In [1]:
import cv2
import mediapipe as mp
import time
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()
cap = cv2.VideoCapture('PoseMotionVideo.mkv')
pTime = 0
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    # print(results.pose_landmarks)
    if results.pose_landmarks:
        mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = img.shape
            print(id, lm)
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 0), 3)
    cv2.imshow("Image", img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

0 x: 0.11274325102567673
y: 0.6279799938201904
z: -0.22893095016479492
visibility: 0.9967932105064392

1 x: 0.10623333603143692
y: 0.6450442671775818
z: -0.22676195204257965
visibility: 0.9964742064476013

2 x: 0.10234931111335754
y: 0.6441124677658081
z: -0.22678199410438538
visibility: 0.9955517649650574

3 x: 0.09848335385322571
y: 0.6430925726890564
z: -0.2268865555524826
visibility: 0.9957835674285889

4 x: 0.11618909984827042
y: 0.6481581926345825
z: -0.22840870916843414
visibility: 0.9973633885383606

5 x: 0.11996710300445557
y: 0.6496925354003906
z: -0.22844940423965454
visibility: 0.9967268705368042

6 x: 0.12372592091560364
y: 0.6512296199798584
z: -0.22859206795692444
visibility: 0.997413694858551

7 x: 0.09396776556968689
y: 0.6368292570114136
z: -0.16871675848960876
visibility: 0.9922738671302795

8 x: 0.12789860367774963
y: 0.6478729844093323
z: -0.17757083475589752
visibility: 0.9964147806167603

9 x: 0.10631421953439713
y: 0.6117274761199951
z: -0.20255330204963684
visi

In [1]:
import cv2
import mediapipe as mp
import time
import math
class poseDetector():
    def __init__(self, mode=False, upBody=False, smooth=True,
                 detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, 1, True, self.upBody, self.smooth,
                                     self.detectionCon, self.trackCon)
    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img
    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList
    def findAngle(self, img, p1, p2, p3, draw=True):
        # Get the landmarks
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]
        # Calculate the Angle
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
        # print(angle)
        # Draw
        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle
def main():
    cap = cv2.VideoCapture('PoseMotionVideo.mkv')
    pTime = 0
    detector = poseDetector()
    while True:
        success, img = cap.read()
        img = detector.findPose(img)
        lmList = detector.findPosition(img, draw=False)
        if len(lmList) != 0:
            print(lmList[14])
            cv2.circle(img, (lmList[14][1], lmList[14][2]), 15, (0, 0, 255), cv2.FILLED)
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 0), 3)
        cv2.imshow("Image", img)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
if __name__ == "__main__":
    main()

[14, 156, 238]
[14, 126, 268]
[14, 103, 274]
[14, 72, 257]
[14, 79, 255]
[14, 78, 254]
[14, 99, 249]
[14, 99, 248]
[14, 94, 246]
[14, 84, 247]
[14, 75, 243]
[14, 76, 241]
[14, 80, 239]
[14, 112, 237]
[14, 70, 239]
[14, 76, 238]
[14, 77, 236]
[14, 88, 237]
[14, 87, 238]
[14, 684, 287]
[14, 682, 297]
[14, 677, 296]
[14, 677, 295]
[14, 676, 295]
[14, 674, 296]
[14, 671, 297]
[14, 668, 299]
[14, 669, 299]
[14, 670, 299]
[14, 671, 299]
[14, 672, 299]
[14, 679, 301]
[14, 687, 300]
[14, 686, 300]
[14, 691, 300]
[14, 700, 300]
[14, 464, 225]
[14, 441, 228]
[14, 437, 230]
[14, 421, 231]
[14, 420, 225]
[14, 427, 226]
[14, 456, 226]
[14, 428, 222]
[14, 461, 227]
[14, 463, 228]
